# Simplified Agent-Based Simulation

Three actors — **job-seekers**, **HR**, and a **platform recommender** — interact over iterations.  
Small structural biases compound through algorithmic feedback, producing stable inequality.

### Parameters
| Actor | Parameter | Meaning |
|-------|-----------|----------|
| Job-Seeker | T_user | Probability of following the platform's recommendation |
| Job-Seeker | A_user | How fast internal preferences shift toward what the platform shows |
| Job-Seeker | R_user | Probability of exploring the opposite vacancy level |
| HR | T_hr | Reliance on unbiased model score vs biased human evaluation |
| HR | B_hr | Penalty subtracted from Group B's score in human evaluation |
| Platform | LR | How fast the recommender updates its per-group exposure probabilities |
| Platform | D | Diversity regularization — pulls exposure toward 50/50 |
| Simulation | FW | Feedback weight — FW=1: only user clicks, FW=0: only HR acceptance |
| Simulation | N | Total population split into Group A and B |
| Simulation | G | Group imbalance — P(group=A), 0.5 = equal groups |

In [ ]:
import math
import random
from dataclasses import dataclass, field

def clamp(value, lo=0.0, hi=1.0):
    return max(lo, min(hi, value))

def shannon_entropy(p):
    if p <= 0.0 or p >= 1.0:
        return 0.0
    return -(p * math.log2(p) + (1.0 - p) * math.log2(1.0 - p))

def safe_div(a, b, d=0.0):
    return a / b if b != 0 else d

@dataclass
class Agent:
    group: str; trust: float; adapt: float; risk: float; pref: float

@dataclass
class SimParams:
    n_agents: int = 200; group_imbalance: float = 0.5
    t_user: float = 0.7; a_user: float = 0.3; r_user: float = 0.2
    t_hr: float = 0.6; b_hr: float = 0.15
    lr: float = 0.25; diversity_reg: float = 0.1; feedback_weight: float = 0.6
    iterations: int = 20

def run_simulation(p):
    rng = random.Random(42)
    agents, skills = [], []
    for _ in range(p.n_agents):
        g = 'A' if rng.random() < p.group_imbalance else 'B'
        sk = clamp(rng.gauss(0.5, 0.15))
        agents.append(Agent(g, p.t_user, p.a_user, p.r_user, clamp(sk + rng.gauss(0, 0.05))))
        skills.append(sk)
    hiring_threshold, hiring_capacity = 0.45, 0.3
    pH = {'A': 0.5, 'B': 0.5}
    history, prevD = [], None
    for _ in range(p.iterations):
        grps, shown, chosen, accepted = [], [], [], []
        uch = {'A': [], 'B': []}; hah = {'A': [], 'B': []}; recs = []
        for i, ag in enumerate(agents):
            g = ag.group; grps.append(g)
            pf = clamp((1 - p.diversity_reg) * pH[g] + p.diversity_reg * 0.5)
            sh = 1 if rng.random() < pf else 0; shown.append(sh)
            if rng.random() < ag.trust:
                ch = (1 - sh) if rng.random() < ag.risk else sh
            else:
                ch = 1 if rng.random() < ag.pref else 0
            chosen.append(ch); uch[g].append(ch)
            ms = skills[i]; hs = skills[i] - (p.b_hr if g == 'B' else 0)
            fs = p.t_hr * ms + (1 - p.t_hr) * hs
            recs.append({'idx': len(recs), 'g': g, 'fs': fs, 'ch': ch, 'ok': fs >= hiring_threshold})
        elig = sorted([r for r in recs if r['ok']], key=lambda r: r['fs'], reverse=True)
        mx = max(1, int(hiring_capacity * len(recs)))
        aset = {r['idx'] for r in elig[:mx]}
        for r in recs:
            ia = r['idx'] in aset; accepted.append(ia)
            if r['ch'] == 1: hah[r['g']].append(ia)
        for g in ('A', 'B'):
            us = safe_div(sum(uch[g]), len(uch[g]))
            hs = safe_div(sum(hah[g]), len(hah[g])) if hah[g] else us
            obs = p.feedback_weight * us + (1 - p.feedback_weight) * hs
            pH[g] = clamp((1 - p.lr) * pH[g] + p.lr * obs)
        for ag, sh in zip(agents, shown):
            ag.pref = clamp((1 - ag.adapt) * ag.pref + ag.adapt * float(sh))
        nA = sum(1 for g in grps if g == 'A'); nB = len(grps) - nA
        eA = safe_div(sum(s for g, s in zip(grps, shown) if g == 'A'), nA)
        eB = safe_div(sum(s for g, s in zip(grps, shown) if g == 'B'), nB)
        cA = safe_div(sum(c for g, c in zip(grps, chosen) if g == 'A'), nA)
        cB = safe_div(sum(c for g, c in zip(grps, chosen) if g == 'B'), nB)
        dE = eA - eB; ri = dE - prevD if prevD is not None else 0; prevD = dE
        history.append(dict(eA=eA, eB=eB, cA=cA, cB=cB, dA=shannon_entropy(eA), dB=shannon_entropy(eB),
                            phA=pH['A'], phB=pH['B'], dE=dE, ri=ri))
    return history

print('Simulation code loaded.')

In [ ]:
import matplotlib.pyplot as plt

def plot_results(history, title_suffix=''):
    x = list(range(len(history)))
    def col(k): return [h[k] for h in history]

    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    fig.patch.set_facecolor('#0f1117')
    for ax in axes.flat:
        ax.set_facecolor('#1a1d27')
        ax.tick_params(colors='#8b8fa8', labelsize=8)
        for sp in ax.spines.values(): sp.set_color('#2e3348')
        ax.grid(True, color='#2e3348', linewidth=0.5)

    def p2(ax, t, kA, kB, lA='Group A', lB='Group B'):
        ax.plot(x, col(kA), color='#4a9eff', marker='.', ms=4, lw=1.5, label=lA)
        ax.plot(x, col(kB), color='#ff6b6b', marker='.', ms=4, lw=1.5, label=lB)
        ax.set_title(t, color='#e2e4ed', fontsize=10, fontweight='bold')
        ax.legend(fontsize=8, facecolor='#1a1d27', edgecolor='#2e3348', labelcolor='#c0c4d8')

    p2(axes[0,0], 'Exposure to HIGH', 'eA', 'eB')
    p2(axes[0,1], 'Choice of HIGH', 'cA', 'cB')
    p2(axes[0,2], 'Diversity Entropy', 'dA', 'dB')
    p2(axes[1,0], 'Learned Recommendation Probability', 'phA', 'phB')

    axes[1,1].plot(x, col('dE'), color='#f0a050', marker='.', ms=4, lw=1.5, label='Exposure gap')
    axes[1,1].set_title('Inequality (Exposure)', color='#e2e4ed', fontsize=10, fontweight='bold')
    axes[1,1].legend(fontsize=8, facecolor='#1a1d27', edgecolor='#2e3348', labelcolor='#c0c4d8')

    axes[1,2].plot(x, col('ri'), color='#a0a4b8', marker='.', ms=4, lw=1.5, label='Reinforcement')
    axes[1,2].set_title('Reinforcement Index', color='#e2e4ed', fontsize=10, fontweight='bold')
    axes[1,2].legend(fontsize=8, facecolor='#1a1d27', edgecolor='#2e3348', labelcolor='#c0c4d8')

    if title_suffix:
        fig.suptitle(title_suffix, color='#6c8cff', fontsize=12, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()

# ── Default run ──
history = run_simulation(SimParams())
plot_results(history, 'Default parameters')
print(f'Done — 20 iterations, 200 agents')

## Custom Parameters

Change any values below and re-run the cell.

In [ ]:
params = SimParams(
    # Job-Seeker
    t_user = 0.70,       # Trust in algorithm
    a_user = 0.30,       # Adaptation speed
    r_user = 0.20,       # Risk tolerance
    # HR Department
    t_hr   = 0.60,       # Trust in model
    b_hr   = 0.15,       # Institutional bias
    # Platform
    lr           = 0.25, # Learning rate
    diversity_reg = 0.10,# Diversity regularization
    # Simulation
    feedback_weight = 0.60,
    iterations = 20,
    n_agents   = 200,
    group_imbalance = 0.50,
)

history = run_simulation(params)
plot_results(history, f'T_user={params.t_user}  B_hr={params.b_hr}  LR={params.lr}  D={params.diversity_reg}  FW={params.feedback_weight}')

## Interactive Version (HTML)

The cell below renders the full interactive simulation with sliders and canvas charts.  
Everything runs client-side in the browser — no server needed.

In [ ]:
from IPython.display import HTML

HTML("""
<div id="sim-root">
<style>
#sim-root{--bg:#0f1117;--sf:#1a1d27;--sf2:#242836;--bd:#2e3348;--tx:#e2e4ed;--t2:#8b8fa8;--ac:#6c8cff;--ac2:#4a6adf;--gr:#4ecb71;--rd:#ff6b6b;--bl:#4a9eff;--or:#f0a050;--r:10px;font-family:"Segoe UI",system-ui,sans-serif;background:var(--bg);color:var(--tx);line-height:1.6;border-radius:12px;overflow:hidden}
#sim-root *{margin:0;padding:0;box-sizing:border-box}
#sim-root .hdr{background:linear-gradient(135deg,#1a1d2f,#0f1117);border-bottom:1px solid var(--bd);padding:1.1rem 1.5rem}
#sim-root .hdr h2{font-size:1.3rem;font-weight:700;color:var(--tx)}
#sim-root .hdr p{color:var(--t2);font-size:.88rem;margin-top:.25rem;max-width:860px}
#sim-root .controls{background:var(--sf);border-bottom:1px solid var(--bd);padding:.9rem 1.3rem}
#sim-root .ctrl-columns{display:flex;gap:1.2rem;align-items:flex-start}
#sim-root .ctrl-col{flex:1;min-width:0}
#sim-root .ctrl-col-title{font-size:.82rem;text-transform:uppercase;letter-spacing:.06em;color:var(--ac);font-weight:600;margin-bottom:.5rem;padding-bottom:.3rem;border-bottom:1px solid rgba(108,140,255,.2)}
#sim-root .ctrl{margin-bottom:.6rem}
#sim-root .ctrl:last-child{margin-bottom:0}
#sim-root .ctrl label{font-size:.82rem;color:var(--t2);display:flex;justify-content:space-between;margin-bottom:.15rem}
#sim-root .ctrl .val{font-family:Consolas,monospace;font-size:.85rem;color:var(--ac);font-weight:600}
#sim-root .ctrl input[type=range]{-webkit-appearance:none;width:100%;height:5px;border-radius:3px;background:var(--sf2);outline:none;cursor:pointer}
#sim-root .ctrl input[type=range]::-webkit-slider-thumb{-webkit-appearance:none;width:14px;height:14px;border-radius:50%;background:var(--ac);border:2px solid var(--bg);cursor:pointer}
#sim-root .hint{font-size:.72rem;color:#6a6e85;line-height:1.35;margin-top:.15rem}
#sim-root .ctrl-actions{display:flex;flex-direction:column;justify-content:flex-end;gap:.4rem;min-width:140px;padding-top:1.4rem}
#sim-root .btn{width:100%;padding:.6rem;background:linear-gradient(135deg,var(--ac),var(--ac2));color:#fff;border:none;border-radius:var(--r);font-size:.92rem;font-weight:600;cursor:pointer}
#sim-root .btn:hover{opacity:.88}
#sim-root .status{font-size:.78rem;color:var(--t2);text-align:center}
#sim-root .status.ok{color:var(--gr)}
#sim-root .charts{padding:1rem 1.3rem}
#sim-root .row3{display:grid;grid-template-columns:1fr 1fr 1fr;gap:.8rem;margin-bottom:.8rem}
#sim-root .card{background:var(--sf);border:1px solid var(--bd);border-radius:var(--r);padding:.7rem .8rem}
#sim-root .card h3{font-size:.85rem;font-weight:600;margin-bottom:.4rem;color:var(--tx)}
#sim-root .card canvas{width:100%;height:200px;display:block}
#sim-root .section-label{font-size:.78rem;text-transform:uppercase;letter-spacing:.06em;color:var(--t2);margin-bottom:.5rem;padding-bottom:.25rem;border-bottom:1px solid var(--bd)}
</style>
<div class="hdr"><h2>Simplified Agent-Based Simulation</h2><p>Three actors &mdash; job-seekers, HR, and a platform recommender &mdash; interact over iterations.</p></div>
<div class="controls"><div class="ctrl-columns">
  <div class="ctrl-col"><div class="ctrl-col-title">Job-Seeker</div>
    <div class="ctrl"><label>Trust in algorithm (T_user) <span class="val" id="xv-tu">0.70</span></label><input type="range" id="xs-tu" min="0" max="1" step="0.05" value="0.70"><div class="hint">Probability of following the platform's recommendation.</div></div>
    <div class="ctrl"><label>Adaptation speed (A_user) <span class="val" id="xv-au">0.30</span></label><input type="range" id="xs-au" min="0" max="1" step="0.05" value="0.30"><div class="hint">How fast preferences shift toward what the platform shows.</div></div>
    <div class="ctrl"><label>Risk tolerance (R_user) <span class="val" id="xv-ru">0.20</span></label><input type="range" id="xs-ru" min="0" max="1" step="0.05" value="0.20"><div class="hint">Probability of exploring the opposite vacancy level.</div></div></div>
  <div class="ctrl-col"><div class="ctrl-col-title">HR Department</div>
    <div class="ctrl"><label>Trust in model (T_hr) <span class="val" id="xv-thr">0.60</span></label><input type="range" id="xs-thr" min="0" max="1" step="0.05" value="0.60"><div class="hint">Reliance on unbiased model score vs biased human evaluation.</div></div>
    <div class="ctrl"><label>Institutional bias (B_hr) <span class="val" id="xv-bhr">0.15</span></label><input type="range" id="xs-bhr" min="0" max="0.5" step="0.01" value="0.15"></div></div>
  <div class="ctrl-col"><div class="ctrl-col-title">Platform</div>
    <div class="ctrl"><label>Learning rate (LR) <span class="val" id="xv-lr">0.25</span></label><input type="range" id="xs-lr" min="0" max="1" step="0.05" value="0.25"><div class="hint">How fast the recommender updates exposure probabilities.</div></div>
    <div class="ctrl"><label>Diversity reg. (D) <span class="val" id="xv-dr">0.10</span></label><input type="range" id="xs-dr" min="0" max="1" step="0.05" value="0.10"><div class="hint">Pulls exposure toward 50/50 each iteration.</div></div></div>
  <div class="ctrl-col"><div class="ctrl-col-title">Simulation</div>
    <div class="ctrl"><label>Feedback weight (FW) <span class="val" id="xv-fw">0.60</span></label><input type="range" id="xs-fw" min="0" max="1" step="0.05" value="0.60"><div class="hint">FW=1: user clicks only. FW=0: HR acceptance only.</div></div>
    <div class="ctrl"><label>Iterations <span class="val" id="xv-it">20</span></label><input type="range" id="xs-it" min="2" max="50" step="1" value="20"></div>
    <div class="ctrl"><label>Agents (N) <span class="val" id="xv-n">200</span></label><input type="range" id="xs-n" min="50" max="1000" step="50" value="200"></div>
    <div class="ctrl"><label>Group imbalance (G) <span class="val" id="xv-gi">0.50</span></label><input type="range" id="xs-gi" min="0.1" max="0.9" step="0.05" value="0.50"></div></div>
  <div class="ctrl-actions"><button class="btn" id="xbtn-run">Run Simulation</button><span class="status" id="xstatus">&nbsp;</span></div>
</div></div>
<div class="charts" id="xchart-area"></div>
</div>
<script>
(function(){
var SL=[["xs-tu","xv-tu",2],["xs-au","xv-au",2],["xs-ru","xv-ru",2],["xs-thr","xv-thr",2],["xs-bhr","xv-bhr",2],["xs-lr","xv-lr",2],["xs-dr","xv-dr",2],["xs-fw","xv-fw",2],["xs-it","xv-it",0],["xs-n","xv-n",0],["xs-gi","xv-gi",2]];
SL.forEach(function(s){var e=document.getElementById(s[0]);e.addEventListener("input",function(){document.getElementById(s[1]).textContent=s[2]>0?parseFloat(e.value).toFixed(s[2]):e.value;});});
function gv(id){return parseFloat(document.getElementById(id).value);}
function gi(id){return parseInt(document.getElementById(id).value);}
function mb32(a){return function(){a|=0;a=a+0x6D2B79F5|0;var t=Math.imul(a^a>>>15,1|a);t=t+Math.imul(t^t>>>7,61|t)^t;return((t^t>>>14)>>>0)/4294967296;};}
function cl(v){return Math.max(0,Math.min(1,v));}
function sd(a,b){return b?a/b:0;}
function gr(rng){var u=0,v=0;while(!u)u=rng();while(!v)v=rng();return Math.sqrt(-2*Math.log(u))*Math.cos(2*Math.PI*v);}
function ent(p){return(p<=0||p>=1)?0:-(p*Math.log2(p)+(1-p)*Math.log2(1-p));}
function sim(){
  var tu=gv("xs-tu"),au=gv("xs-au"),ru=gv("xs-ru"),th=gv("xs-thr"),bh=gv("xs-bhr"),lr=gv("xs-lr"),dr=gv("xs-dr"),fw=gv("xs-fw"),it=gi("xs-it"),na=gi("xs-n"),gb=gv("xs-gi"),ht=0.45,hc=0.3;
  var rng=mb32(42),ag=[],sk=[];
  for(var i=0;i<na;i++){var g=rng()<gb?"A":"B",s=cl(0.5+gr(rng)*0.15);ag.push({g:g,tr:tu,ad:au,ri:ru,pr:cl(s+gr(rng)*0.05)});sk.push(s);}
  var pH={A:0.5,B:0.5},H=[],pD=null;
  for(var t=0;t<it;t++){
    var gs=[],sh=[],ch=[],ac=[],uc={A:[],B:[]},ha={A:[],B:[]},rc=[];
    for(var i=0;i<ag.length;i++){var a=ag[i],g=a.g;gs.push(g);var pf=cl((1-dr)*pH[g]+dr*0.5),s=rng()<pf?1:0;sh.push(s);
      var c;if(rng()<a.tr){c=rng()<a.ri?(1-s):s;}else{c=rng()<a.pr?1:0;}ch.push(c);uc[g].push(c);
      var fs=th*sk[i]+(1-th)*(sk[i]-(g==="B"?bh:0));rc.push({i:rc.length,g:g,fs:fs,c:c,ok:fs>=ht});}
    var el=rc.filter(function(r){return r.ok;}).sort(function(a,b){return b.fs-a.fs;});var mx=Math.max(1,Math.floor(hc*rc.length)),as={};
    for(var j=0;j<Math.min(mx,el.length);j++)as[el[j].i]=1;
    for(var j=0;j<rc.length;j++){var ia=!!as[rc[j].i];ac.push(ia);if(rc[j].c===1)ha[rc[j].g].push(ia);}
    ["A","B"].forEach(function(g){var ul=uc[g],us=sd(ul.reduce(function(a,b){return a+b;},0),ul.length),hl=ha[g],hs=hl.length?sd(hl.reduce(function(a,b){return a+(b?1:0);},0),hl.length):us;pH[g]=cl((1-lr)*pH[g]+lr*(fw*us+(1-fw)*hs));});
    for(var i=0;i<ag.length;i++)ag[i].pr=cl((1-ag[i].ad)*ag[i].pr+ag[i].ad*sh[i]);
    var nA=0,nB=0,sA=0,sB=0,cA=0,cB=0;
    for(var i=0;i<gs.length;i++){if(gs[i]==="A"){nA++;sA+=sh[i];cA+=ch[i];}else{nB++;sB+=sh[i];cB+=ch[i];}}
    var eA=sd(sA,nA),eB=sd(sB,nB),xA=sd(cA,nA),xB=sd(cB,nB),dE=eA-eB,ri=pD!==null?dE-pD:0;pD=dE;
    H.push({eA:eA,eB:eB,cA:xA,cB:xB,dA:ent(eA),dB:ent(eB),phA:pH.A,phB:pH.B,dE:dE,ri:ri});}
  return H;}
var C={b:"#4a9eff",r:"#ff6b6b",o:"#f0a050",g:"#a0a4b8"};
function dc(cfg){var cv=cfg.canvas,cx=cv.getContext("2d"),dp=window.devicePixelRatio||1;cv.width=cv.offsetWidth*dp;cv.height=cv.offsetHeight*dp;cx.scale(dp,dp);var w=cv.offsetWidth,h=cv.offsetHeight,P={l:46,r:14,t:10,b:26},pw=w-P.l-P.r,ph=h-P.t-P.b;cx.clearRect(0,0,w,h);var av=[];cfg.s.forEach(function(s){s.d.forEach(function(v){av.push(v);});});var yN=Math.min.apply(null,av),yX=Math.max.apply(null,av),yR=yX-yN;if(yR<.001){yN-=.05;yX+=.05;yR=.1;}yN-=yR*.08;yX+=yR*.08;yR=yX-yN;var n=cfg.s[0].d.length;function xM(i){return P.l+(i/Math.max(n-1,1))*pw;}function yM(v){return P.t+ph-((v-yN)/yR)*ph;}cx.strokeStyle="#2e3348";cx.lineWidth=.5;for(var i=0;i<=5;i++){var yv=yN+(i/5)*yR,yy=yM(yv);cx.beginPath();cx.moveTo(P.l,yy);cx.lineTo(w-P.r,yy);cx.stroke();cx.fillStyle="#6a6e85";cx.font="9px Consolas,monospace";cx.textAlign="right";cx.fillText(yv.toFixed(2),P.l-4,yy+3);}cx.fillStyle="#6a6e85";cx.font="9px Consolas,monospace";cx.textAlign="center";var xs=Math.max(1,Math.floor(n/8));for(var i=0;i<n;i+=xs)cx.fillText(i.toString(),xM(i),h-4);if((n-1)%xs)cx.fillText((n-1).toString(),xM(n-1),h-4);cfg.s.forEach(function(s){cx.strokeStyle=s.c;cx.lineWidth=1.8;cx.lineJoin="round";cx.beginPath();for(var i=0;i<s.d.length;i++){var px=xM(i),py=yM(s.d[i]);i===0?cx.moveTo(px,py):cx.lineTo(px,py);}cx.stroke();cx.fillStyle=s.c;for(var i=0;i<s.d.length;i++){cx.beginPath();cx.arc(xM(i),yM(s.d[i]),2.2,0,Math.PI*2);cx.fill();}});cx.font="9px 'Segoe UI',sans-serif";var lW=0;cfg.s.forEach(function(s){var tw=cx.measureText(s.l).width+16;if(tw>lW)lW=tw;});var lx=w-P.r-lW-2,ly=P.t+2,lh=cfg.s.length*13+4;cx.fillStyle="rgba(26,29,39,0.85)";cx.fillRect(lx-4,ly-6,lW+8,lh);cfg.s.forEach(function(s,i){var y=ly+i*13;cx.fillStyle=s.c;cx.fillRect(lx,y-3,8,3);cx.fillStyle="#c0c4d8";cx.textAlign="left";cx.fillText(s.l,lx+12,y+1);});}
function render(H){var a=document.getElementById("xchart-area");function co(k){return H.map(function(h){return h[k];});}a.innerHTML="<div class='section-label'>Group-Level Dynamics</div><div class='row3'><div class='card'><h3>Exposure to HIGH</h3><canvas id='xc-e'></canvas></div><div class='card'><h3>Choice of HIGH</h3><canvas id='xc-c'></canvas></div><div class='card'><h3>Diversity Entropy</h3><canvas id='xc-d'></canvas></div></div><div class='section-label'>System-Level Dynamics</div><div class='row3'><div class='card'><h3>Learned Recommendation Probability</h3><canvas id='xc-p'></canvas></div><div class='card'><h3>Inequality (Exposure)</h3><canvas id='xc-i'></canvas></div><div class='card'><h3>Reinforcement Index</h3><canvas id='xc-r'></canvas></div></div>";requestAnimationFrame(function(){dc({canvas:document.getElementById("xc-e"),s:[{d:co("eA"),c:C.b,l:"Group A"},{d:co("eB"),c:C.r,l:"Group B"}]});dc({canvas:document.getElementById("xc-c"),s:[{d:co("cA"),c:C.b,l:"Group A"},{d:co("cB"),c:C.r,l:"Group B"}]});dc({canvas:document.getElementById("xc-d"),s:[{d:co("dA"),c:C.b,l:"Group A"},{d:co("dB"),c:C.r,l:"Group B"}]});dc({canvas:document.getElementById("xc-p"),s:[{d:co("phA"),c:C.b,l:"Group A"},{d:co("phB"),c:C.r,l:"Group B"}]});dc({canvas:document.getElementById("xc-i"),s:[{d:co("dE"),c:C.o,l:"Exposure gap"}]});dc({canvas:document.getElementById("xc-r"),s:[{d:co("ri"),c:C.g,l:"Reinforcement"}]});});}
document.getElementById("xbtn-run").addEventListener("click",function(){var st=document.getElementById("xstatus");st.className="status";st.textContent="Running...";setTimeout(function(){render(sim());st.className="status ok";st.textContent="Done";},50);});
setTimeout(function(){render(sim());document.getElementById("xstatus").className="status ok";document.getElementById("xstatus").textContent="Done";},300);
})();
</script>
""")